In [1]:
import numpy as np
import pandas as pd

## READ THE DATA

In [2]:
fileName = 'input.txt'

In [3]:
with open(fileName) as openFile:
    instructionArray = [[line.rstrip()[0], line.rstrip()[1:]] for line in openFile]
instructionData = pd.DataFrame(instructionArray, columns=['inst', 'value'])
instructionData.head()

,inst,value
0,L,90
1,F,67
2,R,270
3,W,1
4,R,180


## PART 1

In [4]:
class Ship:
    
    def __init__(self, north, south, east, west, direction):
        self.north = north - south
        self.east = east - west
        self.direction = direction
    
    def GetPosition(self):
        if self.north >= 0:
            north = self.north
            south = 0
        else:
            north = 0
            south = abs(self.north)
        
        if self.east >= 0:
            east = self.east
            west = 0
        else:
            east = 0
            west = abs(self.east)
        
        return (north, south, east, west, self.direction)
    
    def ManhattanDistance(self):
        north, south, east, west, direction = self.GetPosition()
        return north+south+east+west
    
    def Move(self, direction, value):
        value = int(value)
        
        if direction == 'N':
            self.north += value
        elif direction == 'S':
            self.north -= value
        elif direction == 'E':
            self.east += value
        elif direction == 'W':
            self.east -= value
        else:
            print(direction, "is not a valid movement direction.")
        
    def Rotate(self, direction, angle):
        angle = int(angle)
        
        if not angle % 90 == 0:
            print("WARNING: angle", angle, "is not expected.")
        
        dic = {'N':0, 'E':1, 'S':2, 'W':3}
        dicInv = {0:'N', 1:'E', 2:'S', 3:'W'}
        
        if direction == 'R':
            self.direction = dicInv[(dic[self.direction] + int(angle/90))%4]
        elif direction == 'L':
            self.direction = dicInv[(dic[self.direction] - int(angle/90))%4]
        else:
            print(direction, "is not a valid rotation direction.")
    
    def ParseInstruction(self, instruction, value):
        if instruction in ['N', 'S', 'E', 'W']:
            self.Move(instruction, value)
        elif instruction == 'F':
            self.Move(self.direction, value)
        elif instruction in ['R', 'L']:
            self.Rotate(instruction, value)
        else:
            print(instruction, "is not a valid instruction.")

In [5]:
ship = Ship(0,0,0,0,'E')
for i in instructionData.index.values:
    inst = instructionData.loc[i,'inst']
    value = instructionData.loc[i,'value']
    ship.ParseInstruction(inst, value)
    #print(ship.GetPosition())

print("(N, S, E, W, DIR):", ship.GetPosition())
print("Manhattan distance:", ship.ManhattanDistance())

(N, S, E, W, DIR): (24, 0, 0, 1109, 'W')
Manhattan distance: 1133


## PART 2

In [6]:
class ShipWithWaypoint:
    def __init__(self, north, south, east, west, northWP, southWP, eastWP, westWP):
        self.north = north - south
        self.east = east - west
        self.northWP = northWP - southWP
        self.eastWP = eastWP - westWP

    def GetShipPosition(self):
        if self.north >= 0:
            north = self.north
            south = 0
        else:
            north = 0
            south = abs(self.north)

        if self.east >= 0:
            east = self.east
            west = 0
        else:
            east = 0
            west = abs(self.east)

        return (north, south, east, west)

    def GetWaypointPosition(self):
        if self.northWP >= 0:
            north = self.northWP
            south = 0
        else:
            north = 0
            south = abs(self.northWP)

        if self.eastWP >= 0:
            east = self.eastWP
            west = 0
        else:
            east = 0
            west = abs(self.eastWP)

        return (north, south, east, west)
    
    def ManhattanDistance(self):
        north, south, east, west = self.GetShipPosition()
        return north+south+east+west
    
    def MoveWaypoint(self, direction, value):
        value = int(value)
        
        if direction == 'N':
            self.northWP += value
        elif direction == 'S':
            self.northWP -= value
        elif direction == 'E':
            self.eastWP += value
        elif direction == 'W':
            self.eastWP -= value
        else:
            print(direction, "is not a valid waypoint movement direction.")
            
    def MoveShipForward(self, value):
        value = int(value)
        
        self.north += value * self.northWP
        self.east += value * self.eastWP
        
    def RotateWaypoint(self, direction, angle):
        angle = int(angle)
        
        if not angle % 90 == 0:
            print("WARNING: angle", angle, "is not expected.")
        
        for iteration in range(int(angle/90)%4):
            if direction == 'R':
                temp = self.northWP
                self.northWP = (-1) * self.eastWP
                self.eastWP = temp
            elif direction == 'L':
                temp = (-1) * self.northWP
                self.northWP = self.eastWP
                self.eastWP = temp
            else:
                print(direction, "is not a valid rotation direction.")

    def ParseInstruction(self, instruction, value):
        if instruction in ['N', 'S', 'E', 'W']:
            self.MoveWaypoint(instruction, value)
        elif instruction == 'F':
            self.MoveShipForward(value)
        elif instruction in ['R', 'L']:
            self.RotateWaypoint(instruction, value)
        else:
            print(instruction, "is not a valid instruction.")

In [7]:
shipCoords = (0,0,0,0)
waypointCoords = (1,0,10,0)
shipWithWaypoint = ShipWithWaypoint(*shipCoords, *waypointCoords)

for i in instructionData.index.values:
    inst = instructionData.loc[i,'inst']
    value = instructionData.loc[i,'value']
    shipWithWaypoint.ParseInstruction(inst, value)
    #print(ship.GetPosition())

print("(N, S, E, W):", shipWithWaypoint.GetShipPosition())
print("Manhattan distance:", shipWithWaypoint.ManhattanDistance())

(N, S, E, W): (36701, 0, 24352, 0)
Manhattan distance: 61053
